In [ ]:
# 5 min

import abagen
import pandas as pd
import numpy as np
from scipy.stats import zscore 
print ("imports done")

data_ahba = '/home/gchan/kg98_scratch/gchan/SIR_SCZ/data-ahba'
files = abagen.fetch_microarray(donors='all', data_dir=data_ahba)

# external atlases
atlas_header = '/fs03/kg98/gchan/Atlases/AHBA_individual/Schaefer_GC/Schaefer_xfm_AA/'
atlas132_footer = '/parcs/Schaefer100_7net+TianS2.nii'
atlas332_footer = '/parcs/Schaefer300_7net+TianS2.nii'

atlas132 = {}
atlas132['image'] = {
    '9861': atlas_header + 'H0351_2001' + atlas132_footer, 
    '10021': atlas_header + 'H0351_2002' + atlas132_footer, 
    '12876': atlas_header + 'H0351_1009' + atlas132_footer, 
    '14380': atlas_header + 'H0351_1012' + atlas132_footer, 
    '15496': atlas_header + 'H0351_1015' + atlas132_footer, 
    '15697': atlas_header + 'H0351_1016' + atlas132_footer
}
atlas132['info'] = atlas_header + 'Schaefer2018_100Parcels_7Networks_order_Tian_Subcortex_S2_label.csv'

atlas332 = {}
atlas332['image'] = {
    '9861': atlas_header + 'H0351_2001' + atlas332_footer, 
    '10021': atlas_header + 'H0351_2002' + atlas332_footer, 
    '12876': atlas_header + 'H0351_1009' + atlas332_footer, 
    '14380': atlas_header + 'H0351_1012' + atlas332_footer, 
    '15496': atlas_header + 'H0351_1015' + atlas332_footer, 
    '15697': atlas_header + 'H0351_1016' + atlas332_footer
}
atlas332['info'] = atlas_header + 'Schaefer2018_300Parcels_7Networks_order_Tian_Subcortex_S2_label.csv'
# print(atlas['image'].keys(), atlas['image']['9861'], atlas['info'])

abagen.check_atlas(atlas132['image']['9861'], atlas132['info'])
print("S132 atlas get")
abagen.check_atlas(atlas332['image']['9861'], atlas332['info'])
print("S332 atlas get")

expression132 = abagen.get_expression_data(atlas132['image'], atlas132['info'], missing='interpolate', data_dir=data_ahba, n_proc = 6)
print ("S132 expression get")
expression332 = abagen.get_expression_data(atlas332['image'], atlas332['info'], missing='interpolate', data_dir=data_ahba, n_proc = 6)
print ("S332 expression get")

imports done
S132 atlas get
S332 atlas get


/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packages/abagen/probes_.py:66: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  reannotated.loc[:, 'entrez_id'] = reannotated['entrez_id'].astype('int')
/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packages/abagen/allen.py:505: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  microarray[subj] = microarray[subj].append(exp)
/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packages/abagen/allen.py:506: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels = labels.append(lab)
/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packag

S132 expression get


/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packages/abagen/probes_.py:66: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  reannotated.loc[:, 'entrez_id'] = reannotated['entrez_id'].astype('int')
/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packages/abagen/allen.py:505: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  microarray[subj] = microarray[subj].append(exp)
/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packages/abagen/allen.py:506: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels = labels.append(lab)
/scratch/kg98/gchan/conda_envs/sir/lib/python3.8/site-packag

S332 expression get


In [6]:
# Check info for LH indices, and trim for LH
# 100 cort L, 10 subcort L, 100 cort R, 10 subcort R
# atlas_info = pd.read_csv(atlas['info'])
# print(atlas_info[98:112])

expr_s132_l = expression132[0:66]
expr_s332_l = expression332[0:166]
print(expr_s132_l.shape, expr_s332_l.shape)

(66, 15635) (166, 15635)


In [7]:
# iterate through genes with GO annotations
# check in gene expression data for each gene
# if so append to a dict of genes

# a 1d array of genes
# GO_genes = pd.read_csv("./data/GO_annots/GO_Homo-sapiens_UniProt_uniq.csv", header=None)
GO_genes = pd.read_csv("./data/GO_annots/GO_Homo-sapiens_UniProt_annot_uniq.csv", header=None)
gene_expr132 = {}
gene_expr332 = {}
genes = []
for gene in GO_genes.iloc[:,0]:
    try:
        gene_expr132[gene] = expr_s132_l[gene][:]
        gene_expr332[gene] = expr_s332_l[gene][:]
        genes.append(gene)
    except Exception:
        pass
        
out132 = pd.DataFrame(data = gene_expr132)
out132 = out132.fillna(0)
out132_z = out132.apply(zscore)
out332 = pd.DataFrame(data = gene_expr332)
out332 = out332.fillna(0)
out332_z = out332.apply(zscore)

print(out132.shape)
# Write out
out132.to_csv('results/gene_expr_s132_indiv.csv', index=False)
out132_z.to_csv('results/gene_expr_s132_indiv_z.csv', index=False)
out332.to_csv('results/gene_expr_s332_indiv.csv', index=False)
out332_z.to_csv('results/gene_expr_s332_indiv_z.csv', index=False)



(66, 10297)


In [8]:
# a list of all genes with abagen gene expression data
genes_df = pd.DataFrame(genes, columns = ["genes"])
genes_df.to_csv('results/all_annotated_AHBA_GO_genes.csv', index=False)

In [20]:
import nibabel as nib
import nibabel.processing
import nilearn
from nilearn import datasets, plotting, surface
import matplotlib.pyplot as plt
from abagen import images

atlas_img = nib.load("/home/gchan/kg98_scratch/gchan/Atlases/Tian/Schaefer_Tian/reordered/Schaefer2018_100Parcels_7Networks_order_Tian_Subcortex_S2_MNI152NLin6Asym_1mm_reordered.nii");
gene = "GNLY"

gene_header = atlas_img.header.copy()
gene_header['datatype'] = 16
gene_affine = atlas_img.affine.copy()
atlas_data = atlas_img.get_fdata()

gene_data = np.zeros_like(atlas_data)

for roi in np.unique(atlas_data):
    if roi == 0:
        continue
    roi_expr = expression132[gene][roi]
    gene_data[atlas_data == roi] = roi_expr

gene_img = nib.Nifti1Image(gene_data, gene_affine, gene_header)
nib.save(gene_img, gene + '_s132_indiv.nii')